# Infer-6-TrueSkill : Systeme de Classement et Apprentissage en Ligne

**Serie** : Programmation Probabiliste avec Infer.NET (6/13)  
**Duree estimee** : 55 minutes  
**Prerequis** : Infer-5-Skills-IRT

---

## Objectifs

- Comprendre le systeme TrueSkill (Xbox Live)
- Implementer des matchs 1v1 et la mise a jour des skills
- Gerer les matchs nuls
- Maitriser l'apprentissage en ligne (posterieurs -> priors)
- Etendre aux equipes et multi-joueurs

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-5-Skills-IRT](Infer-5-Skills-IRT.ipynb) | [Infer-7-Classification](Infer-7-Classification.ipynb) |

---

## 1. Configuration

Nous chargeons Infer.NET pour implementer le systeme TrueSkill, developpe par Microsoft Research pour Xbox Live. Ce systeme de classement bayesien estime les competences des joueurs a partir des resultats de matchs, tout en quantifiant l'incertitude sur ces estimations.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


In [2]:
// Chargement du helper pour visualiser les graphes de facteurs
#load "FactorGraphHelper.cs"

Console.WriteLine("FactorGraphHelper charge. Graphviz disponible : " + FactorGraphHelper.IsGraphvizAvailable());

FactorGraphHelper charge. Graphviz disponible : True


### Environnement pret

Infer.NET est maintenant charge avec tous les namespaces necessaires pour la programmation probabiliste. Les principaux composants utilises dans ce notebook :

| Namespace | Usage |
|-----------|-------|
| `Microsoft.ML.Probabilistic.Distributions` | Gaussiennes pour modeliser les skills |
| `Microsoft.ML.Probabilistic.Models` | Variables et contraintes du modele |
| `Microsoft.ML.Probabilistic.Algorithms` | Expectation Propagation (EP) |

> **Note technique** : TrueSkill utilise l'algorithme **Expectation Propagation** car les facteurs de comparaison (perf1 > perf2) ne sont pas conjugues avec les Gaussiennes. EP approxime ces facteurs par des Gaussiennes, permettant une inference efficace.

## 2. Introduction a TrueSkill

### Contexte

**TrueSkill** est le systeme de classement developpe par Microsoft Research pour Xbox Live. Il est une generalisation bayesienne du systeme Elo.

### Principe

Chaque joueur a un **skill** (competence) modelise par une distribution Gaussienne :
- **Moyenne (mu)** : estimation du skill
- **Variance (sigma^2)** : incertitude sur cette estimation

### Formulation

$$\text{skill}_i \sim \mathcal{N}(\mu_i, \sigma_i^2)$$

$$\text{performance}_i = \text{skill}_i + \epsilon_i, \quad \epsilon_i \sim \mathcal{N}(0, \beta^2)$$

$$\text{Joueur 1 gagne si} \quad \text{performance}_1 > \text{performance}_2$$

### Parametres par defaut

| Parametre | Valeur | Description |
|-----------|--------|-------------|
| mu_initial | 25 | Skill initial |
| sigma_initial | 25/3 | Incertitude initiale |
| beta | sigma/2 | Ecart-type de la performance |

### Comparaison TrueSkill vs Elo

| Aspect | Elo | TrueSkill |
|--------|-----|-----------|
| **Representation** | Score unique (ex: 1800) | Distribution N(mu, sigma) |
| **Incertitude** | Non modelisee | Capturee par sigma |
| **Convergence** | Lente (K fixe) | Rapide (adaptatif via sigma) |
| **Matchs nuls** | +/- points fixes | Reduction d'incertitude |
| **Equipes** | Moyenne des Elo | Somme des performances |
| **Multi-joueurs** | Decomposition en paires | Natif via contraintes d'ordre |

> **Note historique** : TrueSkill a ete developpe en 2006 par Ralf Herbrich et Thore Graepel chez Microsoft Research. Il est utilise sur Xbox Live depuis 2007 pour le matchmaking de millions de joueurs.

## 3. Modele Deux Joueurs

### Construction du modele etape par etape

Le code suivant construit le modele TrueSkill en quatre phases :

1. **Parametres** : Definition des hyperparametres (mu=25, sigma=8.33, beta=4.17)
2. **Priors** : Chaque joueur commence avec la meme distribution Gaussienne
3. **Performances** : Ajout de bruit pour modeliser la variabilite d'un match
4. **Observation** : Le resultat du match (qui a gagne) est observe

Cette structure separe clairement les **croyances initiales** (priors) des **donnees observees** (resultat du match).

In [3]:
// Parametres TrueSkill
double muInitial = 25.0;
double sigmaInitial = 25.0 / 3.0;
double beta = sigmaInitial / 2.0;  // Variabilite de la performance

// Skills des joueurs (priors)
Variable<double> skill1 = Variable.GaussianFromMeanAndVariance(muInitial, sigmaInitial * sigmaInitial).Named("skill1");
Variable<double> skill2 = Variable.GaussianFromMeanAndVariance(muInitial, sigmaInitial * sigmaInitial).Named("skill2");

// Performances (skill + bruit)
Variable<double> perf1 = Variable.GaussianFromMeanAndVariance(skill1, beta * beta).Named("perf1");
Variable<double> perf2 = Variable.GaussianFromMeanAndVariance(skill2, beta * beta).Named("perf2");

// Resultat du match : Joueur 1 gagne
Variable<bool> joueur1Gagne = (perf1 > perf2).Named("joueur1Gagne");

// Observation : Joueur 1 a effectivement gagne
joueur1Gagne.ObservedValue = true;

Console.WriteLine("Modele TrueSkill deux joueurs defini.");

Modele TrueSkill deux joueurs defini.


### Structure du modele graphique

Le modele TrueSkill peut etre represente comme un graphe de facteurs :

```
skill1 ~ N(25, 8.33^2)     skill2 ~ N(25, 8.33^2)
    |                           |
    v                           v
  perf1 = skill1 + eps       perf2 = skill2 + eps
    |                           |
    +-----------> > <-----------+
                  |
                  v
           joueur1Gagne = true (observe)
```

**Intuition** : L'observation que "joueur 1 gagne" impose la contrainte perf1 > perf2. L'inference propage cette information vers les skills, augmentant skill1 et diminuant skill2.

### Execution de l'inference

Nous allons maintenant executer l'inference pour calculer les posterieurs des skills apres avoir observe que le joueur 1 a gagne. L'algorithme EP va propager l'information du resultat vers les distributions de skill.

In [4]:
// Inference apres le match
InferenceEngine moteur = new InferenceEngine(new ExpectationPropagation());
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur.ShowFactorGraph = true;  // Activer la generation du graphe de facteurs

Gaussian skill1Post = moteur.Infer<Gaussian>(skill1);
Gaussian skill2Post = moteur.Infer<Gaussian>(skill2);

Console.WriteLine("=== Apres un match (Joueur 1 gagne) ===");
Console.WriteLine($"\nAvant le match :");
Console.WriteLine($"  Skill1 = N({muInitial:F1}, {sigmaInitial:F2})");
Console.WriteLine($"  Skill2 = N({muInitial:F1}, {sigmaInitial:F2})");

Console.WriteLine($"\nApres le match :");
Console.WriteLine($"  Skill1 = N({skill1Post.GetMean():F2}, {Math.Sqrt(skill1Post.GetVariance()):F2})");
Console.WriteLine($"  Skill2 = N({skill2Post.GetMean():F2}, {Math.Sqrt(skill2Post.GetVariance()):F2})");

Console.WriteLine($"\nChangement de skill :");
Console.WriteLine($"  Joueur 1 : +{skill1Post.GetMean() - muInitial:F2}");
Console.WriteLine($"  Joueur 2 : {skill2Post.GetMean() - muInitial:F2}");

Compiling model...done.
=== Apres un match (Joueur 1 gagne) ===

Avant le match :
  Skill1 = N(25,0, 8,33)
  Skill2 = N(25,0, 8,33)

Apres le match :
  Skill1 = N(29,21, 7,19)
  Skill2 = N(20,79, 7,19)

Changement de skill :
  Joueur 1 : +4,21
  Joueur 2 : -4,21


### Analyse détaillée du résultat

**Avant le match** : Les deux joueurs sont identiques N(25, 8.33)
**Après le match** : Skill1 = N(29.21, 7.19), Skill2 = N(20.79, 7.19)

**Observations clés** :

| Aspect | Valeur | Explication |
|--------|--------|-------------|
| Gain du gagnant | +4.21 | Augmentation significative |
| Perte du perdant | -4.21 | Symétrique (priors identiques) |
| Réduction sigma | 8.33 → 7.19 | Plus d'info = moins d'incertitude |

**Intuition** : Un match entre deux joueurs de même niveau (selon les priors) est **informatif**. Le gagnant a démontré qu'il est probablement meilleur.

> **Comparaison avec Elo** : Contrairement au système Elo classique qui utilise des changements fixes (±16 points), TrueSkill adapte le changement en fonction de l'**incertitude**. Un joueur avec grand sigma (nouveau) verra son rating changer plus rapidement.

In [5]:
// Visualisation du graphe de facteurs TrueSkill
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_17_21_36.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 25 
 
<!-- node1 -->
 
 node1 
 
 GaussianFromMeanAndVariance 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 mean 
 
<!-- node3 -->
 
 node3 
 
 skill1 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 69,44444444444446 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 variance 
 
<!-- node4 -->
 
 node4 
 
 GaussianFromMeanAndVariance 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 perf1 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 17,361111111111114 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 variance 
 
<!-- node7 -->
 
 node7 
 
 Minus 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 a 
 
<!-- node9 -->
 
 node9 
 
 vdouble10 
 
<!-- node7->node9 -->
 
 node7->node9 
 
 
 
<!-- node8 -->
 
 node8 
 
 perf2 
 
<!-- node8->node7 -->
 
 node8->node7 
 
 
 b 
 
<!-- node16 -->
 
 node16 
 
 IsPositive 
 
<!-- node9->node16 -->
 
 node9->node16 
 
 
 x 
 
<!-- node10 -->
 
 node10 
 
 skill2 
 
<!-- node11 -->
 
 node11 
 
 GaussianFromMeanAndVariance 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 mean 
 
<!-- node11->node8 -->
 
 node11->node8 
 
 
 
<!-- node12 -->
 
 node12 
 
 17,361111111111114 
 
<!-- node12->node11 -->
 
 node12->node11 
 
 
 variance 
 
<!-- node13 -->
 
 node13 
 
 25 
 
<!-- node14 -->
 
 node14 
 
 GaussianFromMeanAndVariance 
 
<!-- node13->node14 -->
 
 node13->node14 
 
 
 mean 
 
<!-- node14->node10 -->
 
 node14->node10 
 
 
 
<!-- node15 -->
 
 node15 
 
 69,44444444444446 
 
<!-- node15->node14 -->
 
 node15->node14 
 
 
 variance 
 
<!-- node17 -->
 
 node17 
 
 True 
 
<!-- node16->node17 -->
 
 node16->node17


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Lecture du graphe de facteurs TrueSkill 1v1

Le graphe ci-dessus montre la structure du modele TrueSkill pour un match a deux joueurs :

**Noeuds de variables (ellipses)** :
- `skill1`, `skill2` : Les competences latentes des joueurs (Gaussiennes)
- `perf1`, `perf2` : Les performances observees pendant le match
- `joueur1Gagne` : Le resultat du match (observe = true)

**Noeuds de facteurs (rectangles)** :
- `GaussianFromMeanAndVariance` : Lie les priors aux skills et les skills aux performances
- `IsGreaterThan` (ou `>`) : La contrainte de comparaison entre performances

**Flux d'information** :
L'algorithme Expectation Propagation (EP) propage des messages le long de ce graphe :
1. L'observation `joueur1Gagne = true` envoie un message vers le facteur de comparaison
2. Ce facteur propage l'information vers `perf1` (augmente) et `perf2` (diminue)
3. Les changements de performance se propagent vers les skills correspondants

> **Note technique** : Le facteur `IsGreaterThan` n'est pas conjugue avec les Gaussiennes. EP l'approxime par des moments de Gaussienne, ce qui explique pourquoi TrueSkill utilise EP plutot que VMP.

## 4. Gestion des Matchs Nuls

### Modele

Un match nul se produit quand la difference de performances est dans un intervalle $[-\epsilon, \epsilon]$.

$$|\text{perf}_1 - \text{perf}_2| < \epsilon \Rightarrow \text{match nul}$$

### Implementation avec Variable.ConstrainBetween

Pour modeliser un match nul, nous utilisons `Variable.ConstrainBetween` qui impose que la difference de performances soit dans un intervalle. Cela remplace la contrainte binaire ">" par une contrainte d'intervalle.

In [6]:
// Modele avec possibilite de match nul

double epsilon = 1.0;  // Marge pour match nul

Variable<double> skillA = Variable.GaussianFromMeanAndVariance(muInitial, sigmaInitial * sigmaInitial).Named("skillA");
Variable<double> skillB = Variable.GaussianFromMeanAndVariance(muInitial, sigmaInitial * sigmaInitial).Named("skillB");

Variable<double> perfA = Variable.GaussianFromMeanAndVariance(skillA, beta * beta).Named("perfA");
Variable<double> perfB = Variable.GaussianFromMeanAndVariance(skillB, beta * beta).Named("perfB");

// Difference de performances
Variable<double> diff = (perfA - perfB).Named("diff");

// Match nul : diff dans [-epsilon, epsilon]
Variable.ConstrainBetween(diff, -epsilon, epsilon);

InferenceEngine moteurNul = new InferenceEngine(new ExpectationPropagation());
moteurNul.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteurNul.ShowFactorGraph = true;  // Activer la generation du graphe de facteurs

Gaussian skillAPostNul = moteurNul.Infer<Gaussian>(skillA);
Gaussian skillBPostNul = moteurNul.Infer<Gaussian>(skillB);

Console.WriteLine("=== Apres un match nul ===");
Console.WriteLine($"\nSkillA = N({skillAPostNul.GetMean():F2}, {Math.Sqrt(skillAPostNul.GetVariance()):F2})");
Console.WriteLine($"SkillB = N({skillBPostNul.GetMean():F2}, {Math.Sqrt(skillBPostNul.GetVariance()):F2})");

Console.WriteLine($"\n=> Les deux joueurs gardent le meme skill moyen");
Console.WriteLine($"   mais l'incertitude diminue (on sait qu'ils sont proches)");

Compiling model...done.
=== Apres un match nul ===

SkillA = N(25,00, 6,46)
SkillB = N(25,00, 6,46)

=> Les deux joueurs gardent le meme skill moyen
   mais l'incertitude diminue (on sait qu'ils sont proches)


### Analyse du match nul

**Résultat** : Les deux joueurs gardent mu=25.00 mais sigma baisse de 8.33 à **6.46**

**Interprétation** :

Un match nul entre joueurs de même niveau prior ne change pas l'estimation moyenne, mais **réduit fortement l'incertitude** car :
- On a observé qu'ils performent de manière similaire
- C'est cohérent avec l'hypothèse qu'ils ont le même skill
- Donc on est plus **confiant** dans cette estimation

**Paradoxe apparent** : Un match "sans résultat" apporte quand même de l'information ! Il confirme que les skills sont proches.

> **Application** : Dans les échecs, une nulle entre deux joueurs de ratings similaires ne change presque pas leurs ratings Elo. Mais avec TrueSkill, leur **incertitude** diminue, ce qui affectera les futurs matchs.

In [7]:
// Visualisation du graphe de facteurs pour le match nul
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_17_23_10.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 25 
 
<!-- node1 -->
 
 node1 
 
 GaussianFromMeanAndVariance 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 mean 
 
<!-- node3 -->
 
 node3 
 
 skillA 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 69,44444444444446 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 variance 
 
<!-- node4 -->
 
 node4 
 
 GaussianFromMeanAndVariance 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 perfA 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 17,361111111111114 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 variance 
 
<!-- node7 -->
 
 node7 
 
 Minus 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 a 
 
<!-- node9 -->
 
 node9 
 
 diff 
 
<!-- node7->node9 -->
 
 node7->node9 
 
 
 
<!-- node8 -->
 
 node8 
 
 perfB 
 
<!-- node8->node7 -->
 
 node8->node7 
 
 
 b 
 
<!-- node16 -->
 
 node16 
 
 IsBetween 
 
<!-- node9->node16 -->
 
 node9->node16 
 
 
 x 
 
<!-- node10 -->
 
 node10 
 
 skillB 
 
<!-- node11 -->
 
 node11 
 
 GaussianFromMeanAndVariance 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 mean 
 
<!-- node11->node8 -->
 
 node11->node8 
 
 
 
<!-- node12 -->
 
 node12 
 
 17,361111111111114 
 
<!-- node12->node11 -->
 
 node12->node11 
 
 
 variance 
 
<!-- node13 -->
 
 node13 
 
 25 
 
<!-- node14 -->
 
 node14 
 
 GaussianFromMeanAndVariance 
 
<!-- node13->node14 -->
 
 node13->node14 
 
 
 mean 
 
<!-- node14->node10 -->
 
 node14->node10 
 
 
 
<!-- node15 -->
 
 node15 
 
 69,44444444444446 
 
<!-- node15->node14 -->
 
 node15->node14 
 
 
 variance 
 
<!-- node19 -->
 
 node19 
 
 vbool1 
 
<!-- node16->node19 -->
 
 node16->node19 
 
 
 
<!-- node17 -->
 
 node17 
 
 -1 
 
<!-- node17->node16 -->
 
 node17->node16 
 
 
 lowerBound 
 
<!-- node18 -->
 
 node18 
 
 1 
 
<!-- node18->node16 -->
 
 node18->node16 
 
 
 upperBound 
 
<!-- node21 -->
 
 node21 
 
 Equal 
 
<!-- node19->node21 -->
 
 node19->node21 
 
 
 B 
 
<!-- node20 -->
 
 node20 
 
 True 
 
<!-- node20->node21 -->
 
 node20->node21 
 
 
 A


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Lecture du graphe de facteurs - Match nul

Ce graphe differe du modele 1v1 par le facteur de contrainte :

**Difference structurelle** :
- `diff` : Variable intermediaire representant `perfA - perfB`
- `ConstrainBetween` : Facteur imposant que `diff` soit dans l'intervalle `[-epsilon, epsilon]`

**Comparaison avec le modele 1v1** :

| Aspect | Match 1v1 | Match nul |
|--------|-----------|-----------|
| Facteur de resultat | `IsGreaterThan` | `ConstrainBetween` |
| Information | Ordre strict | Proximite |
| Effet sur mu | Augmente/diminue | Inchange |
| Effet sur sigma | Reduit | Reduit davantage |

Le facteur `ConstrainBetween` est plus informatif car il impose une double contrainte (borne inf et sup), ce qui explique la reduction plus importante de l'incertitude.

## 5. Apprentissage en Ligne

### Principe

Apres chaque match, les **posterieurs** deviennent les **priors** pour le match suivant.

```
Match 1 : Prior -> Inference -> Posterieur
                                    |
                                    v
Match 2 : Prior (= Posterieur 1) -> Inference -> Posterieur
                                                     |
                                                     v
Match 3 : ...
```

### Implementation de la classe TrueSkillOnline

La classe suivante encapsule la logique d'apprentissage en ligne :

- **Dictionary skills** : Stocke le posterieur actuel de chaque joueur
- **GetSkill()** : Retourne le prior initial si le joueur est nouveau
- **EnregistrerMatch()** : Met a jour les posterieurs apres un match
- **AfficherClassement()** : Affiche le rating conservatif (mu - 3*sigma)

Le **rating conservatif** mu - 3*sigma est la metrique publique utilisee par Xbox Live. Il represente une borne inferieure a 99.7% de confiance sur le vrai skill.

In [8]:
// Classe pour gerer l'apprentissage en ligne

public class TrueSkillOnline
{
    private double muInit;
    private double sigmaInit;
    private double beta;
    private InferenceEngine moteur;
    
    // Skills actuels des joueurs
    private Dictionary<string, Gaussian> skills;
    
    public TrueSkillOnline(double muInit = 25, double sigmaInit = 8.33, double beta = 4.17)
    {
        this.muInit = muInit;
        this.sigmaInit = sigmaInit;
        this.beta = beta;
        this.skills = new Dictionary<string, Gaussian>();
        this.moteur = new InferenceEngine(new ExpectationPropagation());
        this.moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    }
    
    public Gaussian GetSkill(string joueur)
    {
        if (!skills.ContainsKey(joueur))
        {
            skills[joueur] = Gaussian.FromMeanAndVariance(muInit, sigmaInit * sigmaInit);
        }
        return skills[joueur];
    }
    
    public void EnregistrerMatch(string gagnant, string perdant)
    {
        // Priors actuels
        Gaussian priorGagnant = GetSkill(gagnant);
        Gaussian priorPerdant = GetSkill(perdant);
        
        // Modele
        Variable<Gaussian> priorG = Variable.Observed(priorGagnant);
        Variable<Gaussian> priorP = Variable.Observed(priorPerdant);
        
        Variable<double> skillG = Variable.Random<double, Gaussian>(priorG);
        Variable<double> skillP = Variable.Random<double, Gaussian>(priorP);
        
        Variable<double> perfG = Variable.GaussianFromMeanAndVariance(skillG, beta * beta);
        Variable<double> perfP = Variable.GaussianFromMeanAndVariance(skillP, beta * beta);
        
        Variable<bool> resultat = (perfG > perfP);
        resultat.ObservedValue = true;  // Le gagnant a gagne
        
        // Inference
        skills[gagnant] = moteur.Infer<Gaussian>(skillG);
        skills[perdant] = moteur.Infer<Gaussian>(skillP);
    }
    
    public void AfficherClassement()
    {
        var classement = skills.OrderByDescending(kv => kv.Value.GetMean());
        Console.WriteLine("\n=== Classement ===");
        int rang = 1;
        foreach (var kv in classement)
        {
            double mu = kv.Value.GetMean();
            double sigma = Math.Sqrt(kv.Value.GetVariance());
            double conservatif = mu - 3 * sigma;  // TrueSkill rating
            Console.WriteLine($"{rang}. {kv.Key,-10} : mu={mu:F1}, sigma={sigma:F2}, rating={conservatif:F1}");
            rang++;
        }
    }
}

Console.WriteLine("Classe TrueSkillOnline definie.");

Classe TrueSkillOnline definie.


### Architecture de l'apprentissage en ligne

La classe `TrueSkillOnline` implemente le pattern bayesien fondamental :

**Cycle d'apprentissage** :

$$P(\theta | D_{1:n}) \propto P(D_n | \theta) \cdot P(\theta | D_{1:n-1})$$

En pratique :
1. Le **posterieur** apres le match n-1 devient le **prior** pour le match n
2. Chaque match apporte de l'information incrementale
3. Le systeme "n'oublie jamais" mais l'influence des anciens matchs diminue naturellement

> **Avantage computationnel** : Contrairement a un recalcul global, l'apprentissage en ligne a une complexite O(1) par match, permettant de gerer des millions de joueurs en temps reel.

### Simulation d'un tournoi complet

Nous simulons maintenant un petit tournoi de 6 matchs entre 4 joueurs. Observez comment les skills evoluent au fur et a mesure des matchs, et comment le classement emerge des resultats.

In [9]:
// Simulation d'un tournoi

var ts = new TrueSkillOnline();

Console.WriteLine("=== Tournoi TrueSkill ===");

// Serie de matchs
var matchs = new (string, string)[] {
    ("Alice", "Bob"),     // Alice bat Bob
    ("Charlie", "Dave"),  // Charlie bat Dave
    ("Alice", "Charlie"), // Alice bat Charlie
    ("Bob", "Dave"),      // Bob bat Dave
    ("Alice", "Dave"),    // Alice bat Dave
    ("Charlie", "Bob")    // Charlie bat Bob
};

foreach (var (gagnant, perdant) in matchs)
{
    Console.WriteLine($"Match : {gagnant} bat {perdant}");
    ts.EnregistrerMatch(gagnant, perdant);
}

ts.AfficherClassement();

=== Tournoi TrueSkill ===
Match : Alice bat Bob
Compiling model...done.
Match : Charlie bat Dave
Compiling model...done.
Match : Alice bat Charlie
Compiling model...done.
Match : Bob bat Dave
Compiling model...done.
Match : Alice bat Dave
Compiling model...done.
Match : Charlie bat Bob
Compiling model...done.

=== Classement ===
1. Alice      : mu=33,3, sigma=6,01, rating=15,2
2. Charlie    : mu=28,3, sigma=5,58, rating=11,6
3. Bob        : mu=21,7, sigma=5,58, rating=4,9
4. Dave       : mu=16,7, sigma=6,01, rating=-1,3


### Analyse du classement final

**Classement obtenu** : Alice > Charlie > Bob > Dave

| Joueur | V-D | Rating | Sigma |
|--------|-----|--------|-------|
| Alice | 3-0 | 15.2 | 6.01 |
| Charlie | 2-1 | 11.6 | 5.58 |
| Bob | 1-2 | 4.9 | 5.58 |
| Dave | 0-3 | -1.3 | 6.01 |

**Observations** :

1. **Transitivité respectée** : Alice > Charlie (directement) et Charlie > Bob (directement) implique Alice > Bob
2. **Rating conservatif** : mu - 3σ pénalise les joueurs avec peu de matchs (plus grande incertitude)
3. **Sigma décroît** : Plus de matchs = moins d'incertitude sur le skill

**Pourquoi utiliser mu - 3σ ?**
- Évite de surclasser un joueur chanceux avec peu de matchs
- Avec 3σ, on a ~99.7% de confiance que le vrai skill est supérieur
- Xbox Live utilise cette métrique pour le matchmaking public

## 6. Extension aux Equipes

### Modele

Pour un match par equipes, la performance d'equipe est la somme des performances individuelles.

### Modelisation de la performance d'equipe

Dans ce modele 2v2, la performance d'une equipe est la **somme** des performances individuelles. Ce choix de modelisation implique que :

- Un bon joueur peut "porter" un coequipier plus faible
- La variance de l'equipe augmente avec le nombre de joueurs
- L'attribution du credit est uniforme entre coequipiers

In [10]:
// Modele par equipes (2v2)

// Equipe 1 : Joueurs A et B
Variable<double> skillA2 = Variable.GaussianFromMeanAndVariance(25, 70).Named("skillA2");
Variable<double> skillB2 = Variable.GaussianFromMeanAndVariance(25, 70).Named("skillB2");

// Equipe 2 : Joueurs C et D
Variable<double> skillC = Variable.GaussianFromMeanAndVariance(25, 70).Named("skillC");
Variable<double> skillD = Variable.GaussianFromMeanAndVariance(25, 70).Named("skillD");

// Performances individuelles
Variable<double> perfA2 = Variable.GaussianFromMeanAndVariance(skillA2, 17).Named("perfA2");
Variable<double> perfB2 = Variable.GaussianFromMeanAndVariance(skillB2, 17).Named("perfB2");
Variable<double> perfC2 = Variable.GaussianFromMeanAndVariance(skillC, 17).Named("perfC2");
Variable<double> perfD2 = Variable.GaussianFromMeanAndVariance(skillD, 17).Named("perfD2");

// Performances d'equipe (somme)
Variable<double> perfEquipe1 = (perfA2 + perfB2).Named("perfEquipe1");
Variable<double> perfEquipe2 = (perfC2 + perfD2).Named("perfEquipe2");

// Equipe 1 gagne
Variable<bool> equipe1Gagne = (perfEquipe1 > perfEquipe2).Named("equipe1Gagne");
equipe1Gagne.ObservedValue = true;

InferenceEngine moteurEquipe = new InferenceEngine(new ExpectationPropagation());
moteurEquipe.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteurEquipe.ShowFactorGraph = true;  // Activer la generation du graphe de facteurs

Console.WriteLine("=== Match par equipes (2v2) ===");
Console.WriteLine("Equipe 1 (A+B) bat Equipe 2 (C+D)\n");

Console.WriteLine($"Skill A apres : {moteurEquipe.Infer<Gaussian>(skillA2).GetMean():F2}");
Console.WriteLine($"Skill B apres : {moteurEquipe.Infer<Gaussian>(skillB2).GetMean():F2}");
Console.WriteLine($"Skill C apres : {moteurEquipe.Infer<Gaussian>(skillC).GetMean():F2}");
Console.WriteLine($"Skill D apres : {moteurEquipe.Infer<Gaussian>(skillD).GetMean():F2}");

Console.WriteLine("\n=> Tous les membres de l'equipe gagnante voient leur skill augmenter");

=== Match par equipes (2v2) ===
Equipe 1 (A+B) bat Equipe 2 (C+D)

Compiling model...done.
Skill A apres : 27,99
Skill B apres : 27,99
Skill C apres : 22,01
Skill D apres : 22,01

=> Tous les membres de l'equipe gagnante voient leur skill augmenter


### Analyse du match par equipes

**Resultats** :

| Joueur | Equipe | Skill avant | Skill apres | Delta |
|--------|--------|-------------|-------------|-------|
| A | Gagnante | 25.00 | 27.99 | +2.99 |
| B | Gagnante | 25.00 | 27.99 | +2.99 |
| C | Perdante | 25.00 | 22.01 | -2.99 |
| D | Perdante | 25.00 | 22.01 | -2.99 |

**Observations cles** :

1. **Attribution uniforme** : Chaque membre recoit le meme changement (priors identiques)
2. **Gain plus faible qu'en 1v1** : +2.99 vs +4.21 car l'information est "diluee" entre coequipiers
3. **Probleme de l'attribution** : Impossible de distinguer le "carry" du "porte"

> **Limitation** : TrueSkill basique attribue egalement le credit/blame. Des extensions comme TrueSkill 2 (2018) utilisent des statistiques individuelles (kills, assists) pour une attribution plus fine.

In [11]:
// Visualisation du graphe de facteurs pour le match par equipes 2v2
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_17_25_65.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 25 
 
<!-- node1 -->
 
 node1 
 
 GaussianFromMeanAndVariance 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 mean 
 
<!-- node3 -->
 
 node3 
 
 skillA2 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 70 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 variance 
 
<!-- node4 -->
 
 node4 
 
 GaussianFromMeanAndVariance 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 perfA2 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 17 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 variance 
 
<!-- node7 -->
 
 node7 
 
 Plus 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 a 
 
<!-- node9 -->
 
 node9 
 
 perfEquipe1 
 
<!-- node7->node9 -->
 
 node7->node9 
 
 
 
<!-- node8 -->
 
 node8 
 
 perfB2 
 
<!-- node8->node7 -->
 
 node8->node7 
 
 
 b 
 
<!-- node16 -->
 
 node16 
 
 Minus 
 
<!-- node9->node16 -->
 
 node9->node16 
 
 
 a 
 
<!-- node10 -->
 
 node10 
 
 skillB2 
 
<!-- node11 -->
 
 node11 
 
 GaussianFromMeanAndVariance 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 mean 
 
<!-- node11->node8 -->
 
 node11->node8 
 
 
 
<!-- node12 -->
 
 node12 
 
 17 
 
<!-- node12->node11 -->
 
 node12->node11 
 
 
 variance 
 
<!-- node13 -->
 
 node13 
 
 25 
 
<!-- node14 -->
 
 node14 
 
 GaussianFromMeanAndVariance 
 
<!-- node13->node14 -->
 
 node13->node14 
 
 
 mean 
 
<!-- node14->node10 -->
 
 node14->node10 
 
 
 
<!-- node15 -->
 
 node15 
 
 70 
 
<!-- node15->node14 -->
 
 node15->node14 
 
 
 variance 
 
<!-- node18 -->
 
 node18 
 
 vdouble88 
 
<!-- node16->node18 -->
 
 node16->node18 
 
 
 
<!-- node17 -->
 
 node17 
 
 perfEquipe2 
 
<!-- node17->node16 -->
 
 node17->node16 
 
 
 b 
 
<!-- node34 -->
 
 node34 
 
 IsPositive 
 
<!-- node18->node34 -->
 
 node18->node34 
 
 
 x 
 
<!-- node19 -->
 
 node19 
 
 perfC2 
 
<!-- node20 -->
 
 node20 
 
 Plus 
 
<!-- node19->node20 -->
 
 node19->node20 
 
 
 a 
 
<!-- node20->node17 -->
 
 node20->node17 
 
 
 
<!-- node21 -->
 
 node21 
 
 perfD2 
 
<!-- node21->node20 -->
 
 node21->node20 
 
 
 b 
 
<!-- node22 -->
 
 node22 
 
 skillD 
 
<!-- node23 -->
 
 node23 
 
 GaussianFromMeanAndVariance 
 
<!-- node22->node23 -->
 
 node22->node23 
 
 
 mean 
 
<!-- node23->node21 -->
 
 node23->node21 
 
 
 
<!-- node24 -->
 
 node24 
 
 17 
 
<!-- node24->node23 -->
 
 node24->node23 
 
 
 variance 
 
<!-- node25 -->
 
 node25 
 
 25 
 
<!-- node26 -->
 
 node26 
 
 GaussianFromMeanAndVariance 
 
<!-- node25->node26 -->
 
 node25->node26 
 
 
 mean 
 
<!-- node26->node22 -->
 
 node26->node22 
 
 
 
<!-- node27 -->
 
 node27 
 
 70 
 
<!-- node27->node26 -->
 
 node27->node26 
 
 
 variance 
 
<!-- node28 -->
 
 node28 
 
 skillC 
 
<!-- node29 -->
 
 node29 
 
 GaussianFromMeanAndVariance 
 
<!-- node28->node29 -->
 
 node28->node29 
 
 
 mean 
 
<!-- node29->node19 -->
 
 node29->node19 
 
 
 
<!-- node30 -->
 
 node30 
 
 17 
 
<!-- node30->node29 -->
 
 node30->node29 
 
 
 variance 
 
<!-- node31 -->
 
 node31 
 
 25 
 
<!-- node32 -->
 
 node32 
 
 GaussianFromMeanAndVariance 
 
<!-- node31->node32 -->
 
 node31->node32 
 
 
 mean 
 
<!-- node32->node28 -->
 
 node32->node28 
 
 
 
<!-- node33 -->
 
 node33 
 
 70 
 
<!-- node3


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Lecture du graphe de facteurs - Match par equipes 2v2

Le graphe 2v2 illustre l'extension du modele TrueSkill aux equipes :

**Structure hierarchique** :

```
          skillA2   skillB2          skillC   skillD
             |         |                |        |
             v         v                v        v
          perfA2   perfB2           perfC2   perfD2
              \       /                  \      /
               \     /                    \    /
                v   v                      v  v
             perfEquipe1              perfEquipe2
                   \                      /
                    \                    /
                     v                  v
                   equipe1Gagne (observe = true)
```

**Facteurs d'agregation** :
- `Plus` (ou `+`) : Combine les performances individuelles en performance d'equipe
- `IsGreaterThan` : Compare les performances d'equipe

**Propagation de credit** :
L'information "equipe 1 gagne" se propage :
1. Vers `perfEquipe1` (augmente) et `perfEquipe2` (diminue)
2. Via les facteurs `Plus`, vers chaque performance individuelle
3. Puis vers chaque skill individuel

> **Dilution du signal** : Avec 4 joueurs au lieu de 2, l'information est diluee. Chaque joueur recoit environ la moitie du changement de skill qu'il aurait eu en 1v1. C'est le "probleme d'attribution de credit" inherent aux jeux d'equipe.

## 7. Multi-joueurs (Free-for-all)

### Modele

Pour N joueurs, on decompose le resultat en N-1 comparaisons par paires :
- 1er > 2e > 3e > ... > Ne

### Implementation avec contraintes d'ordre transitives

Pour N joueurs classes du 1er au Neme, nous imposons N-1 contraintes transitives :
- perf[0] > perf[1] (1er bat 2e)
- perf[1] > perf[2] (2e bat 3e)
- ...
- perf[N-2] > perf[N-1] (avant-dernier bat dernier)

Infer.NET resout ce systeme de contraintes simultanement grace a EP.

In [12]:
// Modele multi-joueurs (4 joueurs)
// Resultat : P1 > P2 > P3 > P4

Variable<double>[] skillsMulti = new Variable<double>[4];
Variable<double>[] perfsMulti = new Variable<double>[4];

for (int i = 0; i < 4; i++)
{
    skillsMulti[i] = Variable.GaussianFromMeanAndVariance(25, 70).Named($"skill_P{i+1}");
    perfsMulti[i] = Variable.GaussianFromMeanAndVariance(skillsMulti[i], 17).Named($"perf_P{i+1}");
}

// Contraintes d'ordre : perf1 > perf2 > perf3 > perf4
Variable.ConstrainTrue(perfsMulti[0] > perfsMulti[1]);
Variable.ConstrainTrue(perfsMulti[1] > perfsMulti[2]);
Variable.ConstrainTrue(perfsMulti[2] > perfsMulti[3]);

InferenceEngine moteurMulti = new InferenceEngine(new ExpectationPropagation());
moteurMulti.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteurMulti.ShowFactorGraph = true;  // Activer la generation du graphe de facteurs

Console.WriteLine("=== Course multi-joueurs ===");
Console.WriteLine("Classement : P1 > P2 > P3 > P4\n");

for (int i = 0; i < 4; i++)
{
    Gaussian post = moteurMulti.Infer<Gaussian>(skillsMulti[i]);
    Console.WriteLine($"Joueur {i+1} (position {i+1}) : mu={post.GetMean():F2}, sigma={Math.Sqrt(post.GetVariance()):F2}");
}

=== Course multi-joueurs ===
Classement : P1 > P2 > P3 > P4

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Joueur 1 (position 1) : mu=32,73, sigma=6,42
Joueur 2 (position 2) : mu=27,23, sigma=5,83
Joueur 3 (position 3) : mu=22,77, sigma=5,83
Joueur 4 (position 4) : mu=17,27, sigma=6,42


### Analyse du mode multi-joueurs

**Resultats de la course** :

| Position | Joueur | Skill mu | Sigma | Delta mu |
|----------|--------|----------|-------|----------|
| 1er | P1 | 32.73 | 6.42 | +7.73 |
| 2e | P2 | 27.23 | 5.83 | +2.23 |
| 3e | P3 | 22.77 | 5.83 | -2.23 |
| 4e | P4 | 17.27 | 6.42 | -7.73 |

**Structure mathematique** :

Les contraintes transitives `P1 > P2 > P3 > P4` impliquent :
- Le 1er et le dernier ont les changements les plus extremes
- Les positions intermediaires ont des changements moderes
- Sigma est plus eleve aux extremes (moins d'info directe)

> **Application** : Ce modele est utilise pour les jeux Battle Royale (Fortnite, PUBG) ou les courses (Mario Kart). Le placement complet apporte plus d'information qu'une simple victoire/defaite.

In [13]:
// Visualisation du graphe de facteurs pour le mode multi-joueurs
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_17_26_31.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 25 
 
<!-- node1 -->
 
 node1 
 
 GaussianFromMeanAndVariance 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 mean 
 
<!-- node3 -->
 
 node3 
 
 skill_P1 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 70 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 variance 
 
<!-- node4 -->
 
 node4 
 
 GaussianFromMeanAndVariance 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 perf_P1 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 17 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 variance 
 
<!-- node7 -->
 
 node7 
 
 Minus 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 a 
 
<!-- node9 -->
 
 node9 
 
 vdouble109 
 
<!-- node7->node9 -->
 
 node7->node9 
 
 
 
<!-- node8 -->
 
 node8 
 
 perf_P2 
 
<!-- node8->node7 -->
 
 node8->node7 
 
 
 b 
 
<!-- node13 -->
 
 node13 
 
 Minus 
 
<!-- node8->node13 -->
 
 node8->node13 
 
 
 a 
 
<!-- node42 -->
 
 node42 
 
 IsPositive 
 
<!-- node9->node42 -->
 
 node9->node42 
 
 
 x 
 
<!-- node10 -->
 
 node10 
 
 skill_P2 
 
<!-- node11 -->
 
 node11 
 
 GaussianFromMeanAndVariance 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 mean 
 
<!-- node11->node8 -->
 
 node11->node8 
 
 
 
<!-- node12 -->
 
 node12 
 
 17 
 
<!-- node12->node11 -->
 
 node12->node11 
 
 
 variance 
 
<!-- node15 -->
 
 node15 
 
 vdouble110 
 
<!-- node13->node15 -->
 
 node13->node15 
 
 
 
<!-- node14 -->
 
 node14 
 
 perf_P3 
 
<!-- node14->node13 -->
 
 node14->node13 
 
 
 b 
 
<!-- node19 -->
 
 node19 
 
 Minus 
 
<!-- node14->node19 -->
 
 node14->node19 
 
 
 a 
 
<!-- node35 -->
 
 node35 
 
 IsPositive 
 
<!-- node15->node35 -->
 
 node15->node35 
 
 
 x 
 
<!-- node16 -->
 
 node16 
 
 skill_P3 
 
<!-- node17 -->
 
 node17 
 
 GaussianFromMeanAndVariance 
 
<!-- node16->node17 -->
 
 node16->node17 
 
 
 mean 
 
<!-- node17->node14 -->
 
 node17->node14 
 
 
 
<!-- node18 -->
 
 node18 
 
 17 
 
<!-- node18->node17 -->
 
 node18->node17 
 
 
 variance 
 
<!-- node21 -->
 
 node21 
 
 vdouble111 
 
<!-- node19->node21 -->
 
 node19->node21 
 
 
 
<!-- node20 -->
 
 node20 
 
 perf_P4 
 
<!-- node20->node19 -->
 
 node20->node19 
 
 
 b 
 
<!-- node28 -->
 
 node28 
 
 IsPositive 
 
<!-- node21->node28 -->
 
 node21->node28 
 
 
 x 
 
<!-- node22 -->
 
 node22 
 
 skill_P4 
 
<!-- node23 -->
 
 node23 
 
 GaussianFromMeanAndVariance 
 
<!-- node22->node23 -->
 
 node22->node23 
 
 
 mean 
 
<!-- node23->node20 -->
 
 node23->node20 
 
 
 
<!-- node24 -->
 
 node24 
 
 17 
 
<!-- node24->node23 -->
 
 node24->node23 
 
 
 variance 
 
<!-- node25 -->
 
 node25 
 
 25 
 
<!-- node26 -->
 
 node26 
 
 GaussianFromMeanAndVariance 
 
<!-- node25->node26 -->
 
 node25->node26 
 
 
 mean 
 
<!-- node26->node22 -->
 
 node26->node22 
 
 
 
<!-- node27 -->
 
 node27 
 
 70 
 
<!-- node27->node26 -->
 
 node27->node26 
 
 
 variance 
 
<!-- node29 -->
 
 node29 
 
 vbool14 
 
<!-- node28->node29 -->
 
 node28->node29 
 
 
 
<!-- node31 -->
 
 node31 
 
 Equal 
 
<!-- node29->node31 -->
 
 node29->node31 
 
 
 B 
 
<!-- node30 -->
 
 node30 
 
 True 
 
<!-- node30->node31 -->
 
 node30->node31 
 
 
 A 
 
<!-- node32 -->
 
 node32 
 
 25 
 
<!-- node33 -->
 
 node33 
 



warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Lecture du graphe de facteurs - Multi-joueurs (Free-for-all)

Le graphe multi-joueurs montre une structure en chaine de contraintes :

**Topologie du graphe** :

```
skill_P1 -> perf_P1 -\
                      >-- (P1 > P2)
skill_P2 -> perf_P2 -/                \
                      \                >-- P2 joue 2 roles
skill_P2 -> perf_P2 ---\              /
                        >-- (P2 > P3)
skill_P3 -> perf_P3 ---/
                        \
                         >-- (P3 > P4)
skill_P4 -> perf_P4 ----/
```

**Caracteristiques cles** :

1. **Chaine de comparaisons** : N-1 facteurs `IsGreaterThan` pour N joueurs
2. **Joueurs intermediaires** : P2 et P3 participent a deux comparaisons chacun
3. **Correlation induite** : Les contraintes couplent les variables entre elles

**Propagation EP iterative** :
Le graphe montre pourquoi EP necessite des iterations :
- L'info de P1>P2 affecte P2
- L'info de P2>P3 affecte aussi P2
- Ces deux sources d'info doivent etre reconciliees

> **Complexite computationnelle** : Contrairement au cas 1v1 (solution analytique), le cas multi-joueurs necessite des iterations EP. C'est pourquoi on observe "Iterating: ..." dans la sortie. La complexite est O(N^2) par iteration pour N joueurs.

## 8. Analyse d'Echecs (Elo Bayesien)

### Application aux echecs

Nous appliquons TrueSkill aux echecs avec des parametres adaptes a l'echelle Elo :

| Parametre | Valeur TrueSkill standard | Valeur echecs |
|-----------|---------------------------|---------------|
| mu_initial | 25 | 1500 |
| sigma_initial | 8.33 | 350 |
| beta | 4.17 | 175 |

L'echelle est simplement multipliee par 60 pour correspondre aux ratings Elo traditionnels.

In [14]:
// Simulation de parties d'echecs

var chess = new TrueSkillOnline(muInit: 1500, sigmaInit: 350, beta: 175);

// Donnees historiques simulees
var partiesEchecs = new (string, string)[] {
    ("Magnus", "Fabiano"),
    ("Magnus", "Ian"),
    ("Fabiano", "Ian"),
    ("Magnus", "Fabiano"),
    ("Magnus", "Ian"),
    ("Ian", "Fabiano"),  // Upset!
    ("Magnus", "Fabiano"),
    ("Magnus", "Ian"),
    ("Fabiano", "Ian"),
    ("Magnus", "Ian")
};

Console.WriteLine("=== Classement Echecs (Elo Bayesien) ===");
Console.WriteLine("\nParties :");
foreach (var (g, p) in partiesEchecs)
{
    Console.WriteLine($"  {g} bat {p}");
    chess.EnregistrerMatch(g, p);
}

chess.AfficherClassement();

=== Classement Echecs (Elo Bayesien) ===

Parties :
  Magnus bat Fabiano
Compiling model...done.
  Magnus bat Ian
Compiling model...done.
  Fabiano bat Ian
Compiling model...done.
  Magnus bat Fabiano
Compiling model...done.
  Magnus bat Ian
Compiling model...done.
  Ian bat Fabiano
Compiling model...done.
  Magnus bat Fabiano
Compiling model...done.
  Magnus bat Ian
Compiling model...done.
  Fabiano bat Ian
Compiling model...done.
  Magnus bat Ian
Compiling model...done.

=== Classement ===
1. Magnus     : mu=1923,7, sigma=213,54, rating=1283,1
2. Fabiano    : mu=1344,7, sigma=183,39, rating=794,6
3. Ian        : mu=1217,4, sigma=182,68, rating=669,4


### Analyse du classement echecs

**Resultats** :

| Joueur | Victoires | Defaites | Mu | Sigma | Rating |
|--------|-----------|----------|-----|-------|--------|
| Magnus | 6 | 0 | 1923.7 | 213.54 | 1283.1 |
| Fabiano | 2 | 4 | 1344.7 | 183.39 | 794.6 |
| Ian | 1 | 5 | 1217.4 | 182.68 | 669.4 |

**Observations** :

1. **Magnus domine** : 6-0 avec un gain de +423.7 points (1500 -> 1923.7)
2. **L'upset compte** : Ian bat Fabiano (match 6), ce qui explique pourquoi Ian n'est pas beaucoup plus bas
3. **Sigma decroit** : Plus de matchs = plus de certitude sur le niveau reel

**Comparaison avec le vrai classement FIDE (2024)** :

| Joueur | FIDE Elo | Notre estimation |
|--------|----------|------------------|
| Magnus Carlsen | ~2830 | 1923.7 |
| Fabiano Caruana | ~2800 | 1344.7 |
| Ian Nepomniachtchi | ~2790 | 1217.4 |

> **Note** : Les vrais ecarts sont plus faibles (30-40 points), car notre simulation suppose que Magnus gagne toujours, ce qui est irrealiste au plus haut niveau.

## 9. Exercice : Simuler un Tournoi

### Enonce

Creez un tournoi avec 6 joueurs et simulez 15 matchs aleatoires.
Comparez le classement final aux "vrais skills" que vous aurez definis.

### Indice

- Definissez des skills "vrais" pour chaque joueur
- Simulez le resultat de chaque match en fonction des skills
- Utilisez TrueSkillOnline pour mettre a jour les estimations

### Simulation avec "vrais skills" connus

Cet exercice illustre un cas fondamental en statistique bayesienne : nous connaissons les vrais skills (simulation) et pouvons evaluer la qualite des estimations.

**Protocole experimental** :
1. Definir les vrais skills de 6 joueurs (inconnus du modele)
2. Simuler 15 matchs ou le meilleur joueur gagne plus souvent
3. Comparer les estimations TrueSkill aux vrais skills

Notez que le joueur ne gagne pas toujours meme s'il est meilleur : on ajoute du bruit (+/- 5 points) pour simuler la variance de performance.

In [15]:
// EXERCICE : Tournoi simule

// Vrais skills (inconnus du systeme)
var vraisSkills = new Dictionary<string, double>
{
    ["Elite1"] = 35,
    ["Elite2"] = 32,
    ["Moyen1"] = 25,
    ["Moyen2"] = 24,
    ["Debutant1"] = 18,
    ["Debutant2"] = 15
};

var joueurs = vraisSkills.Keys.ToArray();
var rng = new Random(42);
var tournoi = new TrueSkillOnline();

Console.WriteLine("=== Tournoi Simule ===");
Console.WriteLine("\nVrais skills :");
foreach (var kv in vraisSkills.OrderByDescending(x => x.Value))
    Console.WriteLine($"  {kv.Key}: {kv.Value}");

Console.WriteLine("\nMatchs :");

// 15 matchs aleatoires
for (int m = 0; m < 15; m++)
{
    // Choisir deux joueurs differents
    int i = rng.Next(joueurs.Length);
    int j;
    do { j = rng.Next(joueurs.Length); } while (j == i);
    
    string j1 = joueurs[i];
    string j2 = joueurs[j];
    
    // Simuler le match (le meilleur gagne avec plus de probabilite)
    double perf1 = vraisSkills[j1] + rng.NextDouble() * 10 - 5;  // +/- 5
    double perf2 = vraisSkills[j2] + rng.NextDouble() * 10 - 5;
    
    string gagnant = perf1 > perf2 ? j1 : j2;
    string perdant = perf1 > perf2 ? j2 : j1;
    
    Console.WriteLine($"  {gagnant} bat {perdant}");
    tournoi.EnregistrerMatch(gagnant, perdant);
}

tournoi.AfficherClassement();

Console.WriteLine("\n=> Comparez le classement estime aux vrais skills !");

=== Tournoi Simule ===

Vrais skills :
  Elite1: 35
  Elite2: 32
  Moyen1: 25
  Moyen2: 24
  Debutant1: 18
  Debutant2: 15

Matchs :
  Elite1 bat Debutant1
Compiling model...done.
  Elite2 bat Debutant1
Compiling model...done.
  Elite2 bat Debutant1
Compiling model...done.
  Elite2 bat Moyen1
Compiling model...done.
  Elite1 bat Debutant1
Compiling model...done.
  Elite1 bat Debutant1
Compiling model...done.
  Elite1 bat Debutant1
Compiling model...done.
  Moyen2 bat Debutant1
Compiling model...done.
  Elite2 bat Moyen2
Compiling model...done.
  Moyen2 bat Debutant1
Compiling model...done.
  Elite1 bat Debutant1
Compiling model...done.
  Elite2 bat Debutant1
Compiling model...done.
  Elite1 bat Moyen2
Compiling model...done.
  Moyen1 bat Debutant1
Compiling model...done.
  Elite1 bat Moyen1
Compiling model...done.

=== Classement ===
1. Elite1     : mu=34,3, sigma=5,08, rating=19,1
2. Elite2     : mu=33,8, sigma=5,48, rating=17,4
3. Moyen2     : mu=24,1, sigma=5,44, rating=7,8
4. Moyen

### Analyse de la simulation

**Comparaison vrais skills vs estimations** :

| Joueur | Vrai skill | Skill estimé | Écart |
|--------|------------|--------------|-------|
| Elite1 | 35 | 34.3 | -0.7 ✓ |
| Elite2 | 32 | 33.8 | +1.8 ✓ |
| Moyen1 | 25 | 22.4 | -2.6 |
| Moyen2 | 24 | 24.1 | +0.1 ✓ |
| Debutant1 | 18 | 12.6 | -5.4 |
| Debutant2 | 15 | - | (pas assez de matchs) |

**Observations** :

1. **Élites bien identifiés** : Le modèle distingue clairement le groupe "élite"
2. **Debutant1 sous-estimé** : A joué beaucoup mais toujours perdu (malchance statistique)
3. **Debutant2 absent** : Sans matchs, reste au prior (invisible dans le classement)

> **Leçon** : TrueSkill converge vers les vrais skills avec suffisamment de matchs, mais des séries malchanceuses peuvent biaiser temporairement les estimations. L'incertitude (sigma) capture ce risque.

## 10. Resume

| Concept | Description |
|---------|-------------|
| **TrueSkill** | Systeme de classement bayesien |
| **Skill** | Gaussienne N(mu, sigma^2) |
| **Performance** | Skill + bruit gaussien |
| **Match nul** | Difference de perf dans [-epsilon, epsilon] |
| **Online learning** | Posterieurs -> Priors |
| **Rating conservatif** | mu - 3*sigma |

### Forces de TrueSkill

- Quantification explicite de l'incertitude via sigma
- Convergence rapide pour les nouveaux joueurs (sigma eleve = grands changements)
- Gestion native des equipes et multi-joueurs
- Apprentissage en ligne efficace (O(1) par match)

### Limitations

- Suppose une skill stable dans le temps (pas d'apprentissage du joueur)
- Attribution uniforme dans les equipes
- Sensible au choix des hyperparametres (beta, epsilon)

### Extensions modernes

- **TrueSkill 2** (2018) : Utilise les statistiques in-game pour attribution fine
- **Glicko-2** : Alternative populaire avec "volatilite" (changement de skill)
- **OpenSkill** : Implementation open-source compatible multi-plateformes

> **Pour aller plus loin** : L'article original "TrueSkill: A Bayesian Skill Rating System" (Herbrich et al., 2006) detaille les approximations EP et les calculs de messages.

---

## Prochaine etape

Dans [Infer-7-Classification](Infer-7-Classification.ipynb), nous explorerons :

- La classification bayesienne
- Le Bayes Point Machine
- Les tests cliniques A/B bayesiens